# Imports

In [11]:
import geopandas as gpd
import pandas as pd
import fiona
import os
import matplotlib.pyplot as plt
import folium
from zipfile import ZipFile
import seaborn as sns
import numpy as np
import datetime as dt
from collections import Counter
from dotenv import load_dotenv, find_dotenv
from urllib.parse import urljoin
import requests
from io import StringIO, BytesIO
import datetime as dt

# Data

In [8]:
bike_df_all = pd.read_csv('raw_data/bike_loans.csv', sep=';')#.drop(columns=['Importe', 
                                                              #        'DescripcionImporte', 
                                                              #      'operario'])
bike_df_all['Fecha_Prestamo'] = bike_df_all['Fecha_Prestamo'].astype('datetime64[ns]')
bike_df_all['Fecha_Devolucion'] = bike_df_all['Fecha_Devolucion'].astype('datetime64[ns]')

bike_df_all.dtypes
# bike_df_all = bike_df_all.dropna()
bike_df = bike_df_all[bike_df_all.Fecha_Devolucion <= '31/12/2019']
bike_df.shape

(11764276, 13)

In [2]:
# bike_df.operario.unique()

In [3]:
bike_df.dtypes

Id_Historico_Prestamo               int64
Id_Usuario                          int64
Id_Tag_Bicicleta                   object
Fecha_Prestamo             datetime64[ns]
Fecha_Devolucion           datetime64[ns]
Id_Aparcamiento_Origen              int64
Posicion_Origen                     int64
Id_Aparcamiento_Destino           float64
Posicion_Destino                  float64
Num_Bici_Hist                     float64
dtype: object

# Weather

In [7]:
# API KEY
load_dotenv(find_dotenv())
OWM_API = os.environ.get("OWM_API")

In [32]:
## change data!
range_2019 = pd.DataFrame(pd.date_range('2019-01-02', '2021-05-01', freq='h'), columns=['hour'])
range_2019.tail().hour

20396   2021-04-30 20:00:00
20397   2021-04-30 21:00:00
20398   2021-04-30 22:00:00
20399   2021-04-30 23:00:00
20400   2021-05-01 00:00:00
Name: hour, dtype: datetime64[ns]

In [39]:
tail = range_2019.tail().hour
tail.max()

Timestamp('2021-05-01 00:00:00')

In [37]:
req = 'http://history.openweathermap.org/data/2.5/history/city'
start = range_2019.hour.min().value
inter = range_2019.hour[50].value
end = range_2019.hour.max().value
tail1 = tail.min().value
tail2 = tail.max().value
params = {
    'id':'3674962', # ID of medellin
    'type':'hour',
    'start':str(tail1)[:10], # unix time
    'end':str(tail2)[:10],
    'appid': OWM_API
}

r = requests.get(req, params=params)
r.status_code

200

In [38]:
r.json()

{'message': 'Count: 5',
 'cod': '200',
 'city_id': 3674962,
 'calctime': 0.024393096,
 'cnt': 5,
 'list': [{'dt': 1619812800,
   'main': {'temp': 287.68,
    'feels_like': 287.72,
    'pressure': 1026,
    'humidity': 94,
    'temp_min': 286.48,
    'temp_max': 289.15},
   'wind': {'speed': 1.54, 'deg': 330},
   'clouds': {'all': 75},
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'light rain',
     'icon': '10d'},
    {'id': 301, 'main': 'Drizzle', 'description': 'drizzle', 'icon': '09d'}],
   'rain': {'1h': 0.17}},
  {'dt': 1619816400,
   'main': {'temp': 287.55,
    'feels_like': 287.55,
    'pressure': 1025,
    'humidity': 94,
    'temp_min': 286.48,
    'temp_max': 288.15},
   'wind': {'speed': 1.54, 'deg': 200},
   'clouds': {'all': 90},
   'weather': [{'id': 302,
     'main': 'Drizzle',
     'description': 'heavy intensity drizzle',
     'icon': '09d'}]},
  {'dt': 1619820000,
   'main': {'temp': 289.06,
    'feels_like': 289.93,
    'pressure': 1020,
    'h